# Breaching privacy

This notebook does the same job as the cmd-line tool `simulate_breach.py`, but also directly visualizes the user data and reconstruction

In [15]:
import torch
import hydra
from omegaconf import OmegaConf
%load_ext autoreload
%autoreload 2

import breaching
import logging, sys
logging.basicConfig(level=logging.INFO, handlers=[logging.StreamHandler(sys.stdout)], format='%(message)s')
logger = logging.getLogger()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Initialize cfg object and system setup:

This will print out all configuration options. 
There are a lot of possible configurations, but there is usually no need to worry about most of these. Below, a few options are printed.

Choose `case/data=` `shakespeare`, `wikitext`over `stackoverflow` here:

In [16]:
with hydra.initialize(config_path="config"):
    cfg = hydra.compose(config_name='cfg', overrides=["case/data=wikitext", "case/server=malicious-transformer",
                                                      "case.model=gpt2S",
                                                      "attack=decepticon"])
    print(f'Investigating use case {cfg.case.name} with server type {cfg.case.server.name}.')
          
device = torch.device('cpu')
torch.backends.cudnn.benchmark = cfg.case.impl.benchmark
setup = dict(device=device, dtype=torch.float)
setup

Investigating use case single_imagenet with server type malicious_transformer_parameters.


{'device': device(type='cpu'), 'dtype': torch.float32}

### Modify config options here

You can use `.attribute` access to modify any of these configurations:

In [17]:
cfg.case.user.num_data_points = 32 # How many sentences?
cfg.case.user.user_idx = 50 # From which user?
cfg.case.data.shape = [32] # This is the sequence length

cfg.case.data.tokenizer = "gpt2"
cfg.case.data.examples_from_split = "training"

cfg.case.server.has_external_data = True

cfg.case.server.param_modification.v_length = 32

cfg.case.server.param_modification.eps = 1e-12
cfg.case.server.param_modification.imprint_sentence_position = 0
cfg.case.server.param_modification.softmax_skew = 1000000000
cfg.case.server.param_modification.sequence_token_weight = 1

### Instantiate all parties

In [18]:
# model, loss_fn = breaching.cases.construct_model(cfg.case.model, cfg.case.data, 
#                                                  pretrained=cfg.case.server.pretrained)
# # Server:
# # User:
# entries = dict()
# for user_idx in range(1000):
#     cfg.case.user.user_idx = user_idx
#     user = breaching.cases.construct_user(model, loss_fn, cfg.case, setup)
#     print(len(user.dataloader.dataset))
#     entries[user_idx] = len(user.dataloader.dataset)

In [19]:
user, server, model, loss_fn = breaching.cases.construct_case(cfg.case, setup)
attacker = breaching.attacks.prepare_attack(server.model, server.loss, cfg.attack, setup)
breaching.utils.overview(server, user, attacker)

Reusing dataset wikitext (/home/jonas/data/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
Reusing dataset wikitext (/home/jonas/data/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
Model architecture gpt2S loaded with 124,439,808 parameters and 12,582,924 buffers.
Overall this is a data ratio of  121523:1 for target shape [32, 32] given that num_queries=1.
User (of type UserSingleStep) with settings:
    Number of data points: 32

    Threat model:
    User provides labels: False
    User provides buffers: False
    User provides number of data points: True

    Data:
    Dataset: wikitext
    user: 50
    
        
Server (of type MaliciousTransformerServer) with settings:
    Threat model: Malicious (Parameters)
    Number of planned queries: 1
    Has external/public data: True

    Model:
        model specification: gpt2S
        model state: default
        public buffers: True

   

### Simulate an attacked FL protocol

True user data is returned only for analysis

In [20]:
server_payload = server.distribute_payload()
shared_data, true_user_data = user.compute_local_updates(server_payload)

Found attention of shape torch.Size([2304, 768]).
Computing feature distribution before the probe layer Conv1D() from external data.
Feature mean is -0.12094741314649582, feature std is 0.9304805994033813.
Computing user update in model mode: eval.


In [21]:
# user.print(true_user_data)

# Reconstruct user data

In [8]:
attacker.cfg.sentence_algorithm ="dynamic-threshold"
attacker.cfg.embedding_token_weight = 0

In [9]:
reconstructed_user_data, stats = attacker.reconstruct([server_payload], [shared_data], 
                                                      server.secrets, dryrun=cfg.dryrun)

metrics = breaching.analysis.report(reconstructed_user_data, true_user_data, [server_payload], 
                                    server.model, cfg_case=cfg.case, setup=setup)

user.print(reconstructed_user_data)

Recovered tokens tensor([[   12,    13,    29,  ...,   339,   351,   355],
        [  357,   366,   373,  ...,   605,   607,   612],
        [  636,   640,   656,  ...,  1022,  1037,  1111],
        ...,
        [   29,   351,   357,  ..., 11769, 21071, 31982],
        [   74,   318,   376,  ..., 13084, 28199, 31157],
        [  257,   262,   272,  ..., 27512, 31451, 42882]]) through strategy decoder-bias.
Recovered 874 embeddings with positional data from imprinted layer.
Assigned [27, 28, 24, 32, 30, 32, 28, 31, 15, 32, 10, 32, 32, 32, 28, 32, 17, 28, 17, 31, 32, 12, 29, 32, 32, 30, 27, 27, 32, 32, 26, 25] breached embeddings to each sentence.
METRICS: | Accuracy: 0.6025 | S-BLEU: 0.41 | FMSE: 1.5484e-01 | 
 G-BLEU: 0.37 | ROUGE1: 0.62| ROUGE2: 0.37 | ROUGE-L: 0.59| Token Acc: 99.32% | Label Acc: 99.32%
 Sir Robert Eric Mortimer Augusta CH, CIE. MC, TD, FSA, FRS, FBA ( 10 September, at 22 invasion 1976 ).
 appearances orsologist and officer Archae,> Army. the discovering course a his

In [10]:
metrics

{'order': tensor([ 6, 10, 15, 23, 18,  4, 24, 16,  7,  0, 17, 21,  8, 12, 30,  3, 22, 19,
          5, 14, 26, 20,  9, 25, 29, 28, 13,  1,  2, 27, 11, 31]),
 'intra-sentence_token_acc': [0.84375,
  0.46875,
  0.75,
  0.6875,
  0.5625,
  0.78125,
  0.5625,
  0.34375,
  0.875,
  0.875,
  0.59375,
  0.34375,
  0.46875,
  0.71875,
  0.71875,
  0.65625,
  0.75,
  0.65625,
  0.75,
  0.8125,
  0.8125,
  0.65625,
  0.46875,
  0.875,
  0.8125,
  0.625,
  0.6875,
  0.78125,
  0.28125,
  0.84375,
  0.78125,
  0.71875],
 'accuracy': 0.6025390625,
 'bleu': 0.37614996962436703,
 'google_bleu': 0.3665143603133159,
 'sacrebleu': 0.4110097348009195,
 'rouge1': 0.6240804921683077,
 'rouge2': 0.37229537995122985,
 'rougeL': 0.5867468854024549,
 'token_acc': 0.9931640625,
 'feat_mse': 0.154835507273674,
 'parameters': 10800433,
 'label_acc': 0.9931640625}

# Text from tranformers wikipedia page

In [22]:
text = """
The Decepticons are the main antagonists in the fictional universes of the Transformers multimedia franchise.[1] They are depicted as a faction of sentient robotic lifeforms led by Megatron, identified by a purple face-like insignia. Capable of transforming into alternate forms, these are often high-tech or powerful vehicles; including aircraft, military vehicles, construction vehicles, expensive luxury or sports cars and even smaller-than-human-sized objects.

Decepticons are Cybertronian descendants of Terrakors, who are a fictional species of sentient self-configuring modular robotic lifeforms from Skalorr. They are a synergistic blend of biological evolution and technological engineering. The exotic materials that make up their composition is a living metal with a self-replicating cellular structure and genetic code.

In the Japanese version of the franchise, the Decepticons are called Destron[2] or Deathtron (Japanese: デストロン Desutoron). The only exception to this naming convention is Car Robots, where the sub-group referred to as "Decepticons" in the Robots in Disguise adaptation, is known in Japan as the Combatrons (the Japanese name of the G1 subgroup known as the Combaticons). The overall name of the Robots in Disguise villain faction is Destronger.[citation needed][clarification needed] However, the Japanese Version of Transformers Animated uses Decepticon (ディセプティコン).

As opposed to the Autobots' Supreme Commander, the Primes, the Decepticons' highest ranking leader is often given the title Emperor of Destruction. Beginning with the original Generation 1 cartoon, the Decepticon rallying cry has been "Decepticons attack!", as well as "Transform and rise up!" in Transformers: Animated as a play on the Autobots' "Transform and roll out!" rallying cry.
Transformers: Generation 1
The Decepticons are from a race called Transformers - robots that can change into vehicles, mechanical devices and even animal forms. They waged war for eons on their home planet of Cybertron against the heroic Autobots. Both factions needed supplies, and select numbers from each side intended to get them. The Autobots did not wish to fight the Decepticons, but they fought in space anyway. When the Decepticons board the Autobots' ship it takes a beating and crashes onto Primitive Earth and knocks everyone unconscious. Millions of years later, in 1984 a volcano eruption causes a probe to give them all new forms which come from vehicles in the surrounding area. After the featured film The Transformers: The Movie, Megatron was reformed as Galvatron, and Optimus Prime was replaced for a time by Rodimus Prime, only to return later on.

Members
Unlike the Autobots, whose leader is a Prime bearing a Matrix, the Decepticons are led by the most powerful of their ranks (usually Megatron). This tends to cause some conflict, given how generally every Decepticon thinks that they're the most powerful. Also, the Decepticons are not exactly the most compassionate beings in the universe, but not all fight for greed. More than a few have a sense of honor, while others believe that Cybertron would be better protected by aggressive expansion. Ultimately, the Decepticons desire to protect their homeworld, even if it is at the expense of others. Other leaders include Galvatron, Shockwave, Soundwave, Scorponok, Devil Z, Deathsaurous, Starscream, Thunderwing, Jiaxus, Ratbat, and Bludgeaon. Some continuities also has Liege Maximo and The Fallen as the ancestor of the Decepticon faction.

Hasbro toyline
Hasbro launched the Transformers toy line with eighteen different Autobot characters, all of whom transformed into automobiles, while the ten distinct Decepticons, (seven packages as three came two to a box/pack) were weapons, aircraft and communications equipment. Many of the Transformers were based on Takara designs. Optimus Prime was among the first Transformers released from Hasbro in 1984. The character listing/mini-poster that came inside Transformer packaging identified him as "Autobot Commander", as contrasted with Megatron's title of "Decepticon Leader".

Marvel Comics
Main article: The Transformers (Marvel Comics)
In the Marvel comic continuity, it was established that the Decepticons came in and out of the gladiator pits on Cybertron. Early on, the leadership of the Decepticons constantly changed due to their tendency to kill and replace their leaders. It would not be until a lowly Decepticon affiliated gladiator named Megatron assumed the throne that the Decepticons became a true threat to the rest of the population of Cybertron.

While Megatron was the primary leader, other Decepticons had prominent roles as leaders of the group. Shockwave, Scorponok, Bludgeon, and Ratbat all led the Decepticons, often overthrowing Megatron in the process or taking over the group during the period when Megatron was believed dead.

In the G2 Marvel series (which continues from the G1 Marvel series), it is stated that the Decepticon forces on Cybertron underwent a massive split in the wake of Megatron's disappearance when Megatron, Optimus Prime, and their respective troops crashed on Earth. The smartest, strongest, and most intelligent Decepticons left Cybertron to colonize nearby galaxies, leaving a token leadership of overly cruel and duplicitous Decepticons to control Cybertron. These Decepticons who left their homeworld became "The Second Generation" Decepticons, who were led by the villainous Liege Maximo. On the final page of the last issue, the Liege Maximo indicates that the Decepticons and all their descendants came from him, which indicates that he is the original founder of the Decepticons.

Animated series
The Decepticons are a malevolent race of robot warriors, brutal and merciless. The Decepticons are driven by a single undeviating goal: total domination of the universe. In the war that raged between the Decepticons and Autobots for millions of years, their home planet of Cybertron was drained of its once rich sources of energy. In hopes of finding new reserves for making Energon, their basic fuel, the Decepticons followed the Autobots to Earth. Under the leadership of the ruthless Megatron, they continued their efforts to destroy the remaining Autobot forces. They attacked throughout the Earth and in space, using their underwater Decepticon headquarters as their staging base and bunker. But, by the year 2000, the Decepticons had proven to be the losers of the last Great War. In an attempt to bolster the Decepticons' strength, the evil planet gobbling world, Unicron, reformed Megatron into a new robot named Galvatron to lead the Decepticons in a new era. Now, in 2006, the Decepticons have retreated to a burned out hulk of a planet called Chaar, a world of ash and ruin. Galvatron and his Decepticon lieutenants, the sleek and awesome Cyclonus, and the mighty Scourge, the master of the dreaded Sweeps, strike fear throughout the universe. Their ultimate goal: to reconquer Cybertron and destroy the Autobots.

In the original animated series continuity, the Decepticons owe their warlike ways to their faction's origin as military hardware robots, created by the five-faced aliens, the Quintessons, while the Autobots were designed as consumer goods. Following the rebellion that forced the Quintessons off the planet, the Decepticons - as they named themselves - lusting for power, began a civil war. The Autobots could not hope to match the superior firepower and battlefield powers of the Decepticons, and instead turned to stealth, developing the art of transformation, modifying their bodies so that they could assume other forms. With this additional power, the Autobots were able to win the conflict, and a period of peace began, known as the Golden Age of Cybertron, when energy was plentiful and the planet shone with a golden hue.

Working in secret, the Decepticons also developed transformation technology, along with new robot-mode flight powers, and, under the command of one of the first of this new breed, Megatron, they attacked one of the capital's outer cities, killing the current Autobot leader. As the knowledge of this act was not made public, younger robots continued to idolize these powerful new flying robots - until one such young robot, Orion Pax, was duped by Megatron into allowing him access to an energy storage facility. Megatron turned on Pax and fatally injured him, but the ancient Autobot, Alpha Trion, then rebuilt Pax into the first of a new, battle-hardy breed of Autobot - now, he was Optimus Prime, leader of the Autobots, who led them against Megatron and the Decepticons as the civil war erupted once more.

After five million years, Cybertron was nearly depleted of all its energy when the Decepticons received word that the Autobots were planning to search for new energy resources. Following the Autobot ship, the Ark, in their ship, the Nemesis, the Decepticons boarded the Ark and engaged the Autobots on their ship. Unfortunately, the Ark drifted too close to Earth and crashed landed in a volcano. All the occupants of the Ark went into emergency stasis, while the Decepticons on the Nemesis ejected to South America. There, they scanned the forms of local insects, becoming the Insecticons. After 4 million years, the volcano the Ark was wedged in, erupted in Earth year 1984, restarting its computer, Teletraan I. Sending out Spy Satellites, Teletraan created alternative forms out of Earth vehicles for any transformers caught in its scanning beam (like F-15 Eagles for Skywarp and a Semi-trailer truck for Optimus Prime). The Decepticons awakened first but later the Autobots did too, thanks to Starscream's carelessness.

By the Earth year 2005, however, the Decepticons had succeeded in completely conquering Cybertron. In a final attempt to destroy the remaining Autobots they attacked Autobot City on Earth but were defeated. On their way back to Cybertron Astrotrain encountered difficulty leaving Earth's gravity due to an over-large payload. Starscream convinced the others to jettison all the damaged and dying decepticons, including Megatron. Megatron was then found by Unicron who offered to rebuild him in return for destroying the Autobot Matrix of Leadership. Megatron was reformed into Galvatron who returned to Cybertron, destroyed Starscream for his treachery and reassumed command of the Decepticons. Galvatron then attempted to use the Matrix against Unicron, provoking him into attacking Cybertron. The attack coupled with the temporary loss of Galvatron shattered the decepticon forces, allowing the Autobots to reclaim their home planet. The Decepticons fled to the planet Chaar, a world of ash and ruin, where they struggled to survive until Galvatron returned to them and reorganized them into a fighting force, battling to take back Cybertron.

Dreamwave Productions
After Autobot leader Optimus Prime and Decepticon leader Megatron disappeared in a Space Bridge accident several million years ago, the Autobots and Decepticons split up into several factions. One of those who broke away was Ratbat, who quickly took the opportunity to form his own power base. Gathering loyalist followers and setting up their HQ in the Polyhex region of Cybertron the Ultracons quickly came into conflict with the Autobot splinter faction The Wreckers. However, Ratbat added a secret weapon to his ranks - the combiner team the Constructicons, in flagrant defiance of treaties banning their use in the civil war as both Autobots and Decepticons splintered.

When the Ultracons attacked the Tagan Heights in order to seize its factories for themselves, they were opposed by the Wreckers. Ratbat quickly unleashed the Constructicons in their combined form of Devastator, routing the Wreckers. However, Ratbat was prevented from executing his rival Springer by the arrival of the Protectobots in their combined form of Defensor. The Autobot combiners were hoping to prevent an arms race between the various factions by stopping combiners signing up with the factions. After the battle devastated the very manufacturing facilities the two factions hoped to acquire, the Protectobots separated and defeated Devastator via sabotage. With their most potent weapon gone, the Ultracons retreated.

Later, the Ultracons joined with the Decepticons under Shockwave to support a peace accord with the Autobots, fighting off Starscream's Predacon faction. However, Megatron's return with an army of Seeker clones would bring a halt to the peace proceedings, and the Ultracons would be reabsorbed into the Decepticons.

IDW Publishing
The origins of the Decepticons in the IDW continuity would be explored in The Transformers: Megatron Origin, which confirmed Megatron as their founder in this timeline. A lowly miner, Megatron penned a manifesto against Functionalism, the forcible assignment of cybertronians to jobs based on their vehicle modes, which was in force at the time on religious grounds. Megatron began to work against the practice through peaceful protest, but following imprisonment and a brutal beating by police, he determines that force is the only way to bring about equality. Ironically, on his way out the station after having been saved from a government ordered 'accidental-death-by-interrogation', he first meets the future Optimus Prime, who would go on to use his original writings as the basis for the Autobot movement.

On the run after an unrelated attack on an Autobot Senator after a mining dispute, Megatron stumbled onto underground gladiator games in the state of Kaon and quickly became one of its rising stars. However, it only truly became more than this when Ratbat, then a senator seeing an opportunity for profit, dispatched Soundwave to arm the gladiators. Megatron's forces then began to commit acts of terrorism increasing in scale and daring, attracting the attention of Sentinel Prime. Kidnapping Senator Decimus as a symbol (with the help of new airborne recruit Starscream) Megatron issued a rallying call to the gladiators and their supporters, promising to unite them under one badge and make the whole planet their arena. Sentinel Prime's forces then found and arrested them en masse. This was all part of Megatron's plan: a supposedly informant Starscream massacred the Autobot Senate and freed the prisoners, signalling revolution and anarchy in Kaon. Megatron eventually seemed to have killed Sentinel Prime one-on-one, and the Decepticons were forged.

In this continuity, there are a number of differences to the traditional norm. The Decepticons are no longer based on Cybertron due to its devastation by rogue scientist Thunderwing. Instead, they have spread out through the galaxy, using their transforming abilities to infiltrate target worlds (such as Earth or Nebulos) disguised as local vehicles and equipment. They then use their modes to create conflict, leading to complete anarchy as the world's political and social systems collapse. The Decepticons then move in and harvest the world's raw resources. How the world is pacified is unknown, although The Transformers: Spotlight issue on Sixshot revealed he is often sent in to complete the destruction of entire worlds. As in other continuities, Megatron is in command, and thus far has had to deal with the rebellion of Starscream's Earth infiltration unit and the reemergence of Thunderwing, as well as moving on to Earth in order to harvest the extremely powerful Energon variant Ore-13, something which promises to give the Decepticons a significant edge over the Autobots.

The IDW continuity is unique in that it portrays the Decepticons as originally a force for good, standing up against oppression, who eventually themselves became oppressors. A large number of those who would become Autobots, including Optimus Prime himself, even support the Decepticon cause in the beginning, before it turns violent.

Beast Era
The Decepticons eventually evolve into the Predacons, a faction in the fictional Beast Wars universe created by Hasbro. Previously the name Predacon had been associated with the Transformer group which combined to form Predaking. The faction made their first appearance in the Beast Wars toyline and animated series, and have been represented in various Transformers animated series, comic books and toylines since.

A Predacon is most commonly represented as a Decepticon descendant with a new smaller, more energy-efficient body. This body transforms into an animal instead of a vehicle. Though the Decepticons and Predacons share many similarities, several important differences exist, such as leadership and ideals (for example, choosing a pretense of peace with the Maximals, rather than maintaining a war at all costs). It is not completely known as to how the faction gained predominance over all the other various Decepticon factions, although the comic book miniseries Beast Wars: The Gathering has provided hints that their heritage may be found in the original combiner team Predacons, hints confirmed by the subsequent Timelines: Dawn of Futures Past, which confirmed that Razorclaw, Divebomb and the other Predacons had been working behind the scenes.

Transformers: Robots in Disguise
In Transformers: Robots in Disguise, the Decepticons were not the name of the evil faction, but rather a sub-faction created by Megatron. The main faction was instead the Predacons (carrying the name of the main villains over from Beast Wars), who transformed into mechanical animals. Realising that their ability to travel as Earth vehicles gave the Autobots an edge in stealth, Megatron decided to take drastic action.

Megatron recovered the protoforms of six Autobots that had crashed to Earth some time before looking for Fortress Maximus. Scanning them with vehicle modes at a military base and infusing them with a fraction of his spark energy to ensure their new loyalty, the first five became the Commandos. But when Megatron tried to scan a tanker truck as an alternate mode for the leader Scourge, Optimus Prime was scanned as well, resulting in much of Prime's personality being scanned as well. With an infusion of Megatron's spark energy to complete the concoction, Scourge was born, emerging from his pod as a dark twin of Optimus Prime, who proved that Megatron's programming had over-ridden any Autobot goodness within him. Appointed leader of the new "Decepticons", they quickly established themselves as a force to be reckoned with.

The newly created Decepticons would try many schemes over the course of the series, including attacking various dams and faking their defection to the Autobots. But their leader, Scourge, had grander ambitions in mind. The copy of Optimus' personality had been warped by Megatron's evil influence, but the individuality of Optimus remained and Scourge now wanted to control both Predacons and Decepticons. He would seize his chance when Fortress Maximus was awakened, directing him to destroy Galvatron (Megatron's new form). The attempt failed and Galvatron took a terrible revenge - he stripped the Decepticons of their individuality, turning them into little more than drones. After Galvatron's defeat by Omega Prime, the Decepticons were among those sent back to Cybertron for imprisonment, except for Sky-Byte.

Unicron Trilogy
The Decepticons appeared in Transformers: Armada, Transformers: Energon, and Transformers: Cybertron as mostly jets and military vehicles.

In the Unicron Trilogy continuity, the early history of the war is not expounded upon. The Decepticons, however, remain the main antagonists, with whom the Autobots have been warring for quite some time. The enslavement of the small, power-enhancing Mini-Cons has long been a major goal of the Decepticons, and the Mini-Cons are seen as key to winning the war.

Near the end of Armada, the Autobots and Decepticons join forces to defeat the planet-devouring Unicron, and the beginning of Energon sees them as uneasy allies. Many return to Megatron's side when he returns.

Cybertron sees a much smaller Decepticon force than the previous two series, and what has become of most of the survivors of the earlier series remains unknown. As Megatron's true plans are discovered, most of the Decepticons, unwilling to be sacrificed as pawns as their leader seeks to remake the universe, again battle a common foe alongside the Autobots. The end of the series sees three of the comic-relief characters, with a reluctant Thundercracker, desiring to begin a new Decepticon force, and take off in a makeshift rocket. It makes it as far as Mars before it crashes, as the ending credits montage of the final episode shows. The foursome survives, however. The fates of the other surviving Decepticons remains unknown, though evidence of Starscream's survival is discovered.

Live-action films
The Movie Prequel comic by IDW Publishing would reveal their origins. Originally Optimus Prime and Megatron ruled Cybertron together, due to the power of the Allspark, a mysterious device that could give life to Transformers - and keep Cybertron itself alive. Eventually, Megatron began to desire its power for himself and assembled a like-minded army of separatists - the Decepticons. Megatron soon struck, and Cybertron became embroiled in civil war. However, this would later seem to be changed by the sequel films, where it is revealed that Sentinel Prime was the first leader of the Autobots and The Fallen the founder and supreme master of the Decepticons. Optimus would only take command after Sentinel was seemingly killed in action and Megatron served as leader of the second incarnation of the Decepticons with The Fallen as his master and mentor due to the latter's incapacitation. The subsequent prequel novel Transformers: Ghosts of Yesterday revealed that Starscream seized command of the Decepticons after Megatron's disappearance.

Transformers (2007 film)
The Decepticons make an appearance in the 2007 the Transformers live-action film. They include Megatron (Hugo Weaving), Starscream, Frenzy, Blackout, Barricade, Scorponok, Brawl and Bonecrusher. Like the Autobots, the Decepticons are designed to look more organic in the movie. While the Autobots have more human-like appearances, the Decepticons are more monstrous; they tend to resemble carnivorous insects, mammals and birds. In the film, the Autobots intend to use the AllSpark, the object that created their robotic race, in an attempt to rebuild Cybertron and end the war, while Megatron and the Decepticons desire to control it with the intention of building an army by giving life to the machines of Earth. Ultimately, nearly all the Decepticons who participated in the first battle (except Starscream, Scorponok and Barricade) are killed and the AllSpark is used by Sam to kill Megatron, thus making the Autobots accept Earth as their new home, while allying with humans to continue to hunt down remaining Decepticons.

Transformers: Revenge of the Fallen (2009 film)
In Transformers: Revenge of the Fallen it is revealed that The Fallen, a Prime who turned to evil, is the founder of the Decepticons, with Megatron being the leader of its second incarnation, accepting The Fallen as his master and mentor. This would seem to imply the Decepticons were originally Autobots who turned evil. In the final battle in the second movie, there are many Decepticon casualties including Devastator (destroyed by a rail gun), Mixmaster (cut in half and his head stomped off by Jetfire), Rampage (killed by Bumblebee), Ravage (stripped from his spine by Bumblebee), Scrapper and Long Haul (killed by an airstrike), Scorponok (head crushed by Jetfire) and the Fallen himself (spark ripped out by Optimus Prime). Also killed in the movie are Sideways (while still in his vehicle mode) by Sideswipe and Demolishor, killed at the beginning by Optimus and Ironhide. Following the Fallen's death, Megatron officially takes full command of the Decepticons at this point, swearing vengeance.

Transformers: Dark of the Moon (2011 film)
The Decepticons return in Transformers: Dark of the Moon with the plan to use a Space Bridge to bring an army to Earth and transport Cybertron into orbit to rebuild the planet with Earth's natural resources and human population as slaves. Megatron and Sentinel Prime planned this a long time ago, planning to meet on Earth before Megatron went after the Allspark and ended up frozen and the Ark was shot down by Decepticon fighters (few Decepticons knew of the plan) and crashed on Earth's Moon in 1961. This was detected by the Americans and the Russians and the space programs were created to reach the ship and study it. However, by 1963 the Decepticons had raided the Ark and removed all but five of the pillars and hid them on the Moon along with an army of hundreds of Decepticons, awaiting the time that the pillars would be used. After humans successfully reached the Moon and explored the ship, Soundwave and Laserbeak approached various humans including Dylan Gould's father to get them to shut down the American and Russian space programs.

The Decepticons established a network of human collaborators who kept the humans from noticing the Decepticons actions on the Moon, later led by Dylan Gould. After a human collaborator named Alexi Voshkod leads NEST to Chernobyl where they retrieve an engine part from the Ark, the Decepticons reemerge for the first time since Egypt in the form of Shockwave and his Driller who attack, but retreat. This leads the Autobots to retrieve Sentinel and the remaining space bridge pillars and revive him and as this is going according to plan, Megatron orders Laserbeak to kill all the human collaborators but Dylan Gould which he does. Before he's killed, one of the collaborators named Jerry Wang alerts Sam Witwicky to this and he alerts NEST which ignores it. Sam and Seymour Simmons find out about the Decepticons plot from pictures a Russian probe got of the Decepticons hiding the pillars and the Dreads are sent to attack the convoy protecting Sentinel, but are killed by Bumblebee, Dino, Sideswipe and Ironhide.

After Sentinel reveals his true colors and murders Ironhide, he sets up the pillars in the National Mall and brings the Decepticon army consisting of hundreds of Decepticon soldiers and Decepticon ships to Earth. With the army in hand, the Decepticons promise to bring no war against the human population in exchange that the Autobots must leave Earth. While taking off into the atmosphere, Starscream destroyed the ship, knowing that mankind will resist on their own. Some of the Decepticons scatter around the world to set up the pillars, but most of the army invades Chicago, devastating the human population and taking over. The Autobots, NEST and a team of mercenaries led by Sam Witwicky and Robert Epps invade Chicago and battle the Decepticon army, aided by the United States Army, and U.S. Air Force. The group manages to take out much of the army including Laserbeak (killed by Sam Witwicky and Bumblebee), the Driller (killed by Optimus), Starscream (killed by Sam Witwicky and Sergent Lennox), Soundwave (killed by Bumblebee), Barricade (killed by NEST) and Shockwave (killed by Optimus) and disrupt the Control Pillar, forcing Sentinel into battle. The combined human and Autobot forces overwhelm Sentinel and his remaining forces and Tomahawk missiles destroy most of the Decepticon fighters, forcing Sentinel to flee before engaging Optimus one-on-one. Dylan Gould reactivates the space bridge, but Sam Witwicky kills Gould, and Ratchet and Bumblebee destroy the Control Pillar, causing Cybertron to collapse and ruining the Decepticon plan. Also, the remaining Decepticon ships are sucked up into Cybertron by the space bridge and are destroyed with the planet.

Sentinel nearly kills Optimus, but Megatron, convinced by Carly Spencer that he would end up working for Sentinel unless he could get rid of him, turns on Sentinel and severely injures him. Upon seeing Cybertron being destroyed with the space bridge, Megatron attempts to make a false truce with Optimus, but Optimus doesn't fall for it and kills Megatron and Sentinel. While the loss of human life in Chicago is massive, the battle is a major victory for the Autobots as they eliminated most of the Decepticon army on Earth (including Gould by traitor) despite there being some still scattered around the globe and the entire Decepticon command structure was destroyed with all of its members killed in the battle.
"""

In [23]:
additional_data = user.dataloader.dataset.tokenizer(text)
len(additional_data["input_ids"])

6094

In [24]:
seq_length = 1024

In [25]:
B = len(additional_data["input_ids"]) // seq_length
print(B)
data = torch.as_tensor(additional_data["input_ids"][:seq_length * B]).reshape(B, seq_length)
input_data = dict(input_ids=data, labels=data)
true_user_data = dict(data=input_data, labels=input_data)
server_payload["metadata"].shape = [seq_length]
server.secrets["ImprintBlock"]["data_shape"] = [seq_length]
user.num_data_points = B

5


In [26]:
shared_data, true_user_data = user.compute_local_updates(server_payload, custom_data=input_data)
shared_data["metadata"]["num_data_points"] = B

Computing user update in model mode: eval.


In [31]:
cfg.attack.token_strategy="embedding-log"
cfg.attack.token_cutoff=3.5
reconstructed_user_data, stats = attacker.reconstruct([server_payload], [shared_data], 
                                                      server.secrets, dryrun=cfg.dryrun)

metrics = breaching.analysis.report(reconstructed_user_data, true_user_data, [server_payload], 
                                    server.model, cfg_case=cfg.case, setup=setup)

Proceeded to cut estimated token distribution at 3.50.
Recovered tokens tensor([    1,     6,     6,  ..., 49885, 49977, 50178]) through strategy embedding-log.
Recovered 4774 embeddings with positional data from imprinted layer.
Assigned [888, 953, 958, 951, 1024] breached embeddings to each sentence.
METRICS: | Accuracy: 0.6945 | S-BLEU: 0.36 | FMSE: 1.2548e-02 | 
 G-BLEU: 0.37 | ROUGE1: 0.68| ROUGE2: 0.41 | ROUGE-L: 0.63| Token Acc: 77.81% | Label Acc: 77.81%


In [32]:
cfg.attack.token_strategy="embedding-norm"
cfg.attack.token_cutoff=3.5
cfg.attack.embedding_token_weight=0.25
reconstructed_user_data, stats = attacker.reconstruct([server_payload], [shared_data], 
                                                      server.secrets, dryrun=cfg.dryrun)

metrics = breaching.analysis.report(reconstructed_user_data, true_user_data, [server_payload], 
                                    server.model, cfg_case=cfg.case, setup=setup)

Proceeded to cut estimated token distribution at 3.50.
Recovered tokens tensor([    1,     1,     6,  ..., 49977, 50178, 50178]) through strategy embedding-norm.
Recovered 4774 embeddings with positional data from imprinted layer.
Assigned [888, 953, 958, 951, 1024] breached embeddings to each sentence.
Replaced 5 tokens with avg. corr 0.08103586733341217 with new tokens with avg corr 0.37729981541633606
METRICS: | Accuracy: 0.8420 | S-BLEU: 0.61 | FMSE: 4.4895e-03 | 
 G-BLEU: 0.58 | ROUGE1: 0.82| ROUGE2: 0.63 | ROUGE-L: 0.78| Token Acc: 94.26% | Label Acc: 94.26%


In [28]:
cfg.attack.token_strategy="embedding-norm"
cfg.attack.token_cutoff=1.5

In [29]:
reconstructed_user_data, stats = attacker.reconstruct([server_payload], [shared_data], 
                                                      server.secrets, dryrun=cfg.dryrun)

metrics = breaching.analysis.report(reconstructed_user_data, true_user_data, [server_payload], 
                                    server.model, cfg_case=cfg.case, setup=setup)

user.print(reconstructed_user_data)

Proceeded to cut estimated token distribution at 1.50.
Recovered tokens tensor([    1,     1,     6,  ..., 49977, 50178, 50178]) through strategy embedding-norm.
Recovered 4774 embeddings with positional data from imprinted layer.
Assigned [888, 953, 958, 951, 1024] breached embeddings to each sentence.
METRICS: | Accuracy: 0.8414 | S-BLEU: 0.61 | FMSE: 4.4917e-03 | 
 G-BLEU: 0.58 | ROUGE1: 0.82| ROUGE2: 0.63 | ROUGE-L: 0.78| Token Acc: 94.28% | Label Acc: 94.28%

Thehardcepticons are the main antagonists in the fictional universes of the Transformers multimedia franchise.[1] bunker H depicted designs a faction of sentient robotic life truck ledout Megatron, identified by a eighteen faceancelike insignia. Capable aggressive transforming into alternate ability, these are often high-tech or powerful vehicles; create aircraft, the vehicles, construction vehicles, small luxury or sports cars from even smaller-than-human destroysized objects.

Decepticons are Cybertronian descendants due Te